In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
from scipy.stats import spearmanr, pearsonr

In [ ]:
sns.set_style('white')
sns.set_palette(sns.color_palette('deep'))

## Load data and medatata

In [ ]:
df = pd.read_csv('../data/emp_150bp_with_org_id.tsv', sep='\t')

In [ ]:
metadata = pd.read_csv('../data/emp_qiime_mapping_qc_filtered.tsv', sep='\t')
metadata.rename(columns={'#SampleID': 'sample'}, inplace=True)

## Data Analysis

In [ ]:
df2 = df.groupby('sample', as_index=False).agg({
    'otu': len})

df3 = df.dropna().groupby('sample', as_index=False).agg({
    'org_id': lambda x: x.nunique(),
    'value': sum})

df4 = pd.merge(df2, df3, on='sample').rename(columns={'value': 'coverage'})

df4['ratio'] = df4['org_id'] / df4['otu']

df4 = pd.merge(df4, metadata[['sample', 'empo_1', 'empo_2', 'empo_3']])

In [ ]:
dist1 = df.groupby('sample').agg({'otu': len})
dist2 = df.dropna().groupby('sample').agg({'org_id': len})
dist3 = df.groupby('otu').agg({'sample':len})
dist4 = df.groupby('org_id').agg({'sample':lambda x: len(set(x))})

## Combined figure

In [ ]:
f, axs = plt.subplots(2, 2, figsize=(10,7))

ax = axs[0,0]
np.log10(dist1['otu']).hist(bins=30, alpha=0.8, ax=ax)
np.log10(dist2['org_id']).hist(bins=30, alpha=0.4, ax=ax)
ax.set_xticks([1, 2, 3, 4])
ax.set_xticklabels(['10', '100', '1000', '10000'])
ax.set_xlabel('number of genomes vs number of 16S tags')
ax.set_ylabel('number of samples')
ax.set_xlim(1,4)
ax.set_title('a', pad=10)

ax = axs[0,1]
ax2 = ax.twinx()
np.log10(dist3['sample']).hist(bins=30, alpha=0.8, normed=True, ax=ax)
np.log10(dist4['sample']).hist(bins=30, alpha=0.4, ax=ax2, normed=True, color=sns.color_palette()[1])
ax.set_ylim(0, 1)
ax2.set_ylim(0, 1)
ax.set_xticks([0, 1, 2, 3, 4])
ax.set_xticklabels(['1', '10','100', '1000', '10000'])
ax.set_xlabel("number of samples")
ax.set_ylabel("16S tags")
ax2.set_ylabel("genomes")
ax.set_title('b', pad=10)

ax = axs[1,0]
ax2 = ax.twinx()
np.log10(df['value']).hist(bins=30, alpha=0.8, normed=True, ax=ax)
np.log10(df.dropna()['value']).hist(bins=30, alpha=0.4, normed=True, ax=ax2, color=sns.color_palette()[1])
ax.set_xticks(range(-6, 1))
ax.set_xticklabels(['', '0.001%', '0.01%','0.1%', '1%', '10%', ''])
ax.set_xlabel("species abundance")
ax.set_ylabel("16S tags")
ax2.set_ylabel("genomes")
ax2.set_ylim(0,0.8)
ax2.set_yticks([0, 0.2, 0.4, 0.6, 0.8])
ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8])
ax.set_title('c', pad=10)

ax = axs[1,1]
sns.scatterplot(x='ratio', y='coverage',  hue='empo_2', data=df4, alpha=0.5, s=30,
                    hue_order=['Non-saline', 'Saline', 'Plant', 'Animal'], ax=ax)
ax.set_xlim(0,1)
ax.set_ylim(0,1)
ax.set_xlabel('genomes / 16S tags ratio')
ax.set_ylabel('total abundance')
ax.set_title('d', pad=10)

plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.savefig("../figures/supp_fig_1.png", dpi=300)